In [1]:
#Sample submission with past month most sold products plus the item already sold to the customer:

In [2]:
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm

In [3]:
#read all the data:
items = pd.read_csv("dressipi_recsys2022/item_features.csv")
candidate = pd.read_csv("dressipi_recsys2022/candidate_items.csv")
purchase =  pd.read_csv("dressipi_recsys2022/train_purchases.csv")
sessions =  pd.read_csv("dressipi_recsys2022/train_sessions.csv")
sessions_test = pd.read_csv("dressipi_recsys2022/test_final_sessions.csv")
leaderboard = pd.read_csv("dressipi_recsys2022/test_leaderboard_sessions.csv")

In [4]:
#set the date column to a date format
purchase['date'] = pd.to_datetime(purchase['date'])

In [5]:
max_day = purchase['date'].max() #max date 31/05/2021 
min_day = max_day -datetime.timedelta(31)
cond = purchase['date'] >= min_day
new_purchases = purchase[cond]

In [6]:
#top 99 most sold items in the last month of the training data
top_seller = new_purchases['item_id'].value_counts()[0:100].index.to_list()

In [7]:
leaderboard

,session_id,item_id,date
0,26,19185,2021-06-16 09:53:54.158
1,200,17089,2021-06-25 12:23:40.811
2,200,17089,2021-06-25 12:24:36.631
3,200,8060,2021-06-25 12:24:41.677
4,200,4758,2021-06-25 12:24:50.692
...,...,...,...
229349,4439653,25955,2021-06-11 10:22:57.47
229350,4439653,12179,2021-06-11 10:23:00.663
229351,4439757,2078,2021-06-30 11:42:15.073
229352,4439757,2078,2021-06-30 11:43:13.725


In [8]:
def generate_solution():
    solution = [[]]
    for x in tqdm(leaderboard.session_id.unique()):
        for j in range(100):
            solution.append([int(x),int(top_seller[j]),int(j+1)])
    return solution


In [9]:
solution = generate_solution()
solution.pop(0)
solution = pd.DataFrame(solution, columns = ['session_id','item_id', 'rank']).reset_index(drop = True)

100%|██████████| 50000/50000 [00:04<00:00, 11137.96it/s]


In [10]:
solution_csv = solution.to_csv
solution.to_csv(r'solution_csv.csv',index = False)
#first simple dummy solution:

In [11]:
products_dict = {}
for i,x in enumerate(zip(leaderboard['session_id'], leaderboard['item_id'])):
    session, item = x
    if session not in products_dict:
        products_dict[session] = {}
    
    if item not in products_dict[session]:
        products_dict[session][item] = 0
    
    products_dict[session][item] += 1

In [12]:
def generate_solution2():
    solution = [[]]
    for x in tqdm(leaderboard.session_id.unique()):
        l = []
        if x in products_dict:
            l = sorted((products_dict[x]).items(), key=lambda x: x[1], reverse=True)
            l = [y[0] for y in l]
            
            if len(l)>=101:
                l = l[0:100]
            
            for k in range(len(l)):
                solution.append([int(x),int(l[k]),int(k+1)])
            
        for j in range(len(l),100,1):
            solution.append([int(x),int(top_seller[j]),int(j+1)])
                                
    return solution


In [13]:
solution_2 = generate_solution2()

100%|██████████| 50000/50000 [00:03<00:00, 12761.47it/s]


In [14]:
solution_2.pop(0)
solution_2 = pd.DataFrame(solution_2, columns = ['session_id','item_id','rank'])

In [15]:
solution_2.tail()

,session_id,item_id,rank
4999995,4439757,23038,96
4999996,4439757,19310,97
4999997,4439757,3741,98
4999998,4439757,21100,99
4999999,4439757,19270,100


In [16]:
solution_2_csv = solution_2.to_csv
solution_2.to_csv(r'solution_2.csv',index = False)


In [17]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.python.keras.layers import Input, LSTM, Bidirectional, Dense, Embedding

In [12]:
items_unique = items.item_id.unique()
item_charac = {}
for i in tqdm(items_unique):
    selected_items = items[items.item_id == i]
    llista = []
    for j in range(len(selected_items)):
        llista.append(selected_items.iloc[j,1])
        llista.append(selected_items.iloc[j,2])
    item_charac[i] = llista

100%|██████████| 23691/23691 [00:40<00:00, 578.10it/s]


In [13]:
item_charac[2]

[56,
 365,
 62,
 801,
 68,
 351,
 33,
 802,
 72,
 75,
 29,
 123,
 16,
 38,
 50,
 76,
 61,
 462,
 53,
 6,
 7,
 394,
 69,
 885,
 47,
 123]